In [1]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
from numpy import asarray
#from dnn_app_utils_v3 import *

In [ ]:
def load_proyectddo():
    import os 
    from PIL import Image
    import numpy as np

    train_set_x_orig = []
    # train_set_x_orig.tolist()
    # train_set_x_orig = np.array([])
    # train_set_x_orig = np.array()

    for dirname, _, filenames in os.walk('makeup/train/makeup'):
        for filename in filenames:
            image_path = os.path.join(dirname, filename)
            img = Image.open(image_path)
            image_array = np.array(img)
            #image_array = asarray(img)
            train_set_x_orig.append(image_array)
            
    for dirname, _, filenames in os.walk('makeup/train/no_makeup'):
        for filename in filenames:
            image_path = os.path.join(dirname, filename)
            img = Image.open(image_path)
            image_array = np.array(img)
            #image_array = asarray(img)
            train_set_x_orig.append(image_array)
    
    train_set_y = []
    for i in range(0, 858):
        #makeup
        train_set_y.append(1)
    
    for i in range(0, 361):
        #no makeup
        train_set_y.append(0)
    
    
    
    # train_set_x_orig = np.asarray(train_set_x_orig)
    # train_set_x_orig = np.array(train_set_x_orig)
    return np.array(train_set_x_orig), np.array([train_set_y])

In [ ]:
train_set_x_orig, train_set_y_orig = load_proyectddo()

In [ ]:
print(train_set_x_orig.shape)
print(train_set_y_orig.shape)

In [ ]:
train_set_x_orig

In [ ]:
# Example of a picture
index = 900
plt.imshow(train_set_x_orig[index])

In [ ]:
# Reshape the training and test examples 
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))